In [1]:
import os, os.path, re
import numpy as np
from sklearn.preprocessing import normalize, scale
from collections import Counter
from sys import getsizeof
import time
from sklearn.cluster import *

** Data preprocessing **

Extract filenames and reorder it numerically.

In [2]:
files_path = 'collected_models/'
ts_labels = ['heart_rate', 'oxygen_concentration', 'chest_volume']

number_ts_pieces = len(os.listdir(files_path)) / len(ts_labels)
# if the number is not divided 
if abs(number_ts_pieces - round(number_ts_pieces)) > 0:
    print('ERROR: invalid number of files in ', files_path)
else:
    number_ts_pieces = int(number_ts_pieces)    

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):    
    return [ atoi(c) for c in re.split('(\d+)', text) ]

file_names = sorted(os.listdir(files_path), key=natural_keys)

** Extract models from files **

After the extraction we preprocess them. Eliminate parameters and refactor the variable

In [3]:
def get_models_from_file(number_of_file, type_of_ts):
    index_of_ts = ts_labels.index(type_of_ts)
    file_name = file_names[number_ts_pieces * index_of_ts + number_of_file]        
    file_opened = open(files_path + file_name, 'r')
    func_and_params_names = file_opened.readlines()
    models = []
    for entity in func_and_params_names:          
        entity = entity.split(' ')[-1]
        models.append(entity)
        models[-1] = models[-1].strip()                    
        models[-1] = re.sub(r'X\[(\d+)\]', r'x\1', models[-1])    
    return models

Write functions responsible for conversion of a string-handle to a tree matrix and codings.

In [4]:
def find_number_of_tokens(handle):    
    counter_tokens = 0
    counter_variables = 0

    for i in range(len(handle)):
        if handle[i] == '_': 
            counter_tokens += 1
        elif i < len(handle)-1 and handle[i] == 'x' and handle[i+1].isdigit():
            counter_variables += 1;

    return (counter_tokens, counter_variables)

In [10]:
def create_map_tokens_params():
    file_opened = open('data/numbParam.txt', 'r')
    primitives_lines = file_opened.readlines()
    tokens_codes = {line.split()[0] : int(ind) for ind,line in enumerate(primitives_lines)}    
    tokens_params = {line.split()[0] : int(line.split()[1]) for line in primitives_lines}    
    return (tokens_codes, tokens_params)

In [11]:
def dfs_search_on_handle(handle):
    counters = find_number_of_tokens(handle)
    number_tokens = counters[0] + counters[1]
    
    waiting_tokens = []
    encodings = np.zeros(number_tokens, dtype = int)        
    current_token, left, right = 0, 0, 0
    is_a_token_processed_now = False    
    
    map_tokens_params = create_map_tokens_params()[0]
    
    for right in range(len(handle)):
        if handle[right] == '_':
            # the root is detected
            waiting_tokens.append(current_token)
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            right += 1
            break;  
    
    matr = [[] for i in range(number_tokens)]            
    
    # now process the remaining vertices
    reserved_right = right
    for right in np.arange(right, len(handle)):
        if handle[right] == ')':
            waiting_tokens.pop()        
    
        if not is_a_token_processed_now and handle[right].isalpha():
            is_a_token_processed_now = True
            left = right
    
        # if a token is found
        if handle[right] == '_':
            # new token is detected
            current_token += 1
            matr[waiting_tokens[-1]].append(current_token)
            waiting_tokens.append(current_token)
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            is_a_token_processed_now = False      
        
        # if a variable is found
        if right < len(handle)-1 and handle[right] == 'x' and handle[right+1].isdigit():
            # new variable is detected
            current_token += 1
            matr[waiting_tokens[-1]].append(current_token)
            while right < len(handle)-1 and handle[right] == 'x' and handle[right+1].isdigit():
                right += 1
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            is_a_token_processed_now = False            
    
    return (matr, encodings)

In [12]:
def incidence_to_adjacency(incidence):
    size_of_mat = len(incidence[0])
    adjacency = np.zeros((size_of_mat, size_of_mat))    
    for ind, row in enumerate(incidence[0]):
        adjacency[ind][row] = 1
    return adjacency

** Collect primitive structural features from a population of models**

In [13]:
def get_simple_features_from_ts(number_of_file, type_of_ts, tokens_codes):
    models = get_models_from_file(number_of_file, type_of_ts)
    
    primitive_frequences = np.zeros(len(tokens_codes) + 1)
    lower_bound_code_variables = tokens_codes['x0']

    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        model_primitive_frequences = Counter(encodings)
        for key in model_primitive_frequences:
            primitive_frequences[key] += model_primitive_frequences[key]
        primitive_frequences[-1] += len(encodings)
    primitive_frequences[-1] = primitive_frequences[-1] / len(models)
    #return normalize(primitive_frequences.reshape(-1,1), axis=0)
    return primitive_frequences.reshape(-1,1)

In [14]:
tokens_codes, _ = create_map_tokens_params()
feature_matrices_of_ts = {label : np.zeros((len(tokens_codes) + 1, number_ts_pieces)) for label in ts_labels}

for label in ts_labels:
    for index in range(number_ts_pieces):        
        feature_matrices_of_ts[label][:,index] = get_simple_features_from_ts(index, label, tokens_codes)[:,0]
        
#feature_matrices_of_ts = {label : scale(feature_matrices_of_ts[label].T) for label in ts_labels}        
feature_matrices_of_ts = {label : feature_matrices_of_ts[label].T for label in ts_labels}        

In [15]:
unite_feature_matrix = np.vstack((feature_matrices_of_ts[label] for label in ts_labels))

** Now perform two-class classification **

In [16]:
which_label_is_positive = 'oxygen_concentration'

target_vector = np.zeros((unite_feature_matrix.shape[0],1))
ts_labels_in_order_from_dictionary = [label for label in feature_matrices_of_ts]
index_of_label_positive = ts_labels_in_order_from_dictionary.index(which_label_is_positive)
all_inidices_of_samples = np.arange(target_vector.shape[0])
positive_indices_samples = number_ts_pieces * index_of_label_positive + np.arange(number_ts_pieces)
negative_positive_samples = [ind for ind in all_inidices_of_samples if not ind in positive_indices_samples]
target_vector[number_ts_pieces * index_of_label_positive:number_ts_pieces * (index_of_label_positive + 1)] = np.ones((number_ts_pieces,1))
backup_target = target_vector

Divide set of samples on training and test.

In [17]:
fraction_of_test_samples = 0.3

indices_of_test_sample = np.random.choice([True, False], len(all_inidices_of_samples), p = [fraction_of_test_samples, 1 - fraction_of_test_samples])
target_vector = 2* target_vector - 1 
train_matrix = unite_feature_matrix[~indices_of_test_sample,:]
test_matrix = unite_feature_matrix[indices_of_test_sample,:]
train_target = target_vector[~indices_of_test_sample].reshape(sum(~indices_of_test_sample),)
test_target = target_vector[indices_of_test_sample].reshape(sum(indices_of_test_sample),)

In [18]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(train_matrix, train_target) 
print(clf.n_support_)
predictions = clf.predict(test_matrix)
print("predictions: ", predictions)
errors = predictions != test_target
print("error = ", sum(errors) / len(errors))
print("error on positive = ", sum(errors[test_target == max(test_target)]) / len(errors[test_target == max(test_target)]))
print("error on negative = ", sum(errors[test_target == min(test_target)]) / len(errors[test_target == min(test_target)]))

ValueError: Found array with 0 sample(s) (shape=(0, 37)) while a minimum of 1 is required.

In [73]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(train_matrix, train_target) 
print(clf.n_support_)
predictions = clf.predict(train_matrix)
print("predictions: ", predictions)
errors = predictions != train_target
print("error = ", sum(errors) / len(errors))
print("error on positive = ", sum(errors[train_target == max(train_target)]) / len(errors[train_target == max(train_target)]))
print("error on negative = ", sum(errors[train_target == min(train_target)]) / len(errors[train_target == min(train_target)]))

[40 33]
predictions:  [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
error =  0.314285714286
error on positive =  1.0
error on negative =  0.0


In [63]:
from sklearn import neighbors, datasets



n_neighbors = 1
for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(train_matrix, train_target)

    predictions = clf.predict(np.c_[test_matrix])
    print("weights: ", weights)
    errors = predictions != test_target
    print("error = ", sum(errors) / len(errors))
    print("error on positive = ", sum(errors[test_target == max(test_target)]) / len(errors[test_target == max(test_target)]))
    print("error on negative = ", sum(errors[test_target == min(test_target)]) / len(errors[test_target == min(test_target)]))


weights:  uniform
error =  0.326923076923
error on positive =  0.722222222222
error on negative =  0.117647058824
weights:  distance
error =  0.326923076923
error on positive =  0.722222222222
error on negative =  0.117647058824


In [21]:
from sklearn import neighbors, datasets



h = .02  # step size in the mesh
error = np.inf
where_min = -1

for n_neighbors in 1 + np.arange(30):
    for weights in ['uniform', 'distance']:
        # we create an instance of Neighbours Classifier and fit the data.
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        clf.fit(train_matrix, train_target)

        predictions = clf.predict(np.c_[test_matrix])
        errors = predictions != test_target
        if sum(errors) / len(errors) < error:
            where_min = n_neighbors
            error = sum(errors) / len(errors)
            

where_min, error

(1, 0.32692307692307693)

** Collect DFS-code features **

In [30]:
def levenstein_string_distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n+1) # Keep current and previous row, not entire matrix
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [31]:
def distance_between_populations(population_first, population_second):
    len_first, len_second = len(population_first), len(population_second)
    # calculate sum of distances for all pairs of models: one from first population, the other from second
    cumulative_distance = 0
    for model_from_first in population_first:
        for model_from_second in population_second:
            cumulative_distance = cumulative_distance + levenstein_string_distance(model_from_first, model_from_second)
    cumulative_distance = cumulative_distance / (len_first * len_second)
    return cumulative_distance

In [ ]:
def get_simple_features_from_ts(number_of_file, type_of_ts, tokens_codes):
    models = get_models_from_file(number_of_file, type_of_ts)
    
    primitive_frequences = np.zeros(len(tokens_codes))    

    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        primitive_frequences = encodings

    return primitive_frequences.reshape(-1,1)
    #return primitive_frequences.reshape(-1,1)

In [49]:
def create_matrix_of_codes_of_one_population(number_of_file, type_of_ts, tokens_codes):
    codes = 'QWERTYUIOPASDFGHJKLZXCVBNM123456789/*-+=?!'
    models = get_models_from_file(number_of_file, type_of_ts)
    
    matrix_representation = []    
    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        encodings = np.array(encodings)
        if len(model) == 0:
            break
        matrix_representation.append(''.join(np.array(list(codes))[encodings]))

    return matrix_representation
    #return primitive_frequences.reshape(-1,1)

In [55]:
tokens_codes, _ = create_map_tokens_params()
feature_matrices_of_ts = {label : [] for label in ts_labels}
    
for label in ts_labels:
    for index in range(number_ts_pieces):
        feature_matrices_of_ts[label].append(create_matrix_of_codes_of_one_population(index, label, tokens_codes))

In [82]:
distances_between_segments = np.zeros((len(ts_labels) * number_ts_pieces, len(ts_labels) * number_ts_pieces))


start = time.time()
for ind_label_f,label_f in enumerate(feature_matrices_of_ts):
    for ind_label_s,label_s in enumerate(feature_matrices_of_ts):
        print(label_f, label_s)
        for ind_population_f,population_f in enumerate(feature_matrices_of_ts[label_f]):
            for ind_population_s,population_s in enumerate(feature_matrices_of_ts[label_s]):
                index_first = ind_label_f * number_ts_pieces + ind_population_f
                index_second = ind_label_s * number_ts_pieces + ind_population_s
                distances_between_segments[index_first][index_second] = distance_between_populations(population_f[0:10], population_s[0:10])
    
end = time.time()
print(end - start)

heart_rate heart_rate
heart_rate oxygen_concentration
heart_rate chest_volume
oxygen_concentration heart_rate
oxygen_concentration oxygen_concentration
oxygen_concentration chest_volume
chest_volume heart_rate
chest_volume oxygen_concentration
chest_volume chest_volume
400.93179059028625


In [95]:
random_state = 50
clusters = KMeans(n_clusters=3, random_state=random_state).fit_predict(distances_between_segments)
true_clusters = np.zeros(len(ts_labels) * number_ts_pieces)
for ind,_ in enumerate(ts_labels):
    true_clusters[ind * number_ts_pieces:(ind + 1) * number_ts_pieces] = ind * np.ones(number_ts_pieces)
print(sum(clusters != true_clusters) / len(clusters))
clusters

0.566666666667


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0,
       2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [93]:
random_state = 170
clusters = AgglomerativeClustering(n_clusters=3).fit_predict(distances_between_segments)
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [79]:
l = [0,2,3,4]
l[1:2]

[2]

In [ ]:
fraction_of_test_samples = 0.3

test_indices = np.random.choice([True, False], number_ts_pieces, p = [fraction_of_test_samples, 1 - fraction_of_test_samples])
target_vector = 2* target_vector - 1 

train_matrix = unite_feature_matrix[~indices_of_test_sample,:]
test_matrix = unite_feature_matrix[indices_of_test_sample,:]
train_target = target_vector[~indices_of_test_sample].reshape(sum(~indices_of_test_sample),)
test_target = target_vector[indices_of_test_sample].reshape(sum(indices_of_test_sample),)
